# **NOTEBOOK DE CALCUL DES EMISSIONS SONORES**

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import math
import Emission
import altair as alt

import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import Connexion_Transfert as ct

In [10]:
testR1 = Emission.Route(2000,0.01,90,80)
testR2 = Emission.Route(2000,0.01,80,80)
testR3 = Emission.Route(2000,0.01,90,90)

NiveR1, NiveR2, NiveR3 = testR1.lwm, testR2.lwm, testR3.lwm
NiveR1, NiveR2, NiveR3

(88.62282098426773, 87.64366458271999, 88.62285587472778)

In [4]:
testR1 = Emission.Route(2000,200,50,50, categorieRevt='r1')
testR2 = Emission.Route(2000,200,50,50, categorieRevt='r2')
testR3 = Emission.Route(2000,200,50,50, categorieRevt='r3')

NiveR1, NiveR2, NiveR3 = testR1.lwm, testR2.lwm, testR3.lwm
NiveR1, NiveR2, NiveR3

(84.73892562460821, 86.47172763439141, 87.67362448349147)

In [6]:
tmja, tmjapld=11318, 4928
pc_pl=4928/11138*100
q_vl_d=381
q_pl_d=286
q_mpl_d=286*25
q_hpl_d=286*18.5

In [7]:
pc_pl

44.2449272759921

In [23]:
niveauEmissionNmpn08=Emission.Route(q_vl_d, q_pl_d, 50,50).lwm
niveauEmissionNmpn08Plamade=Emission.Route(q_vl_d, q_mpl_d+q_hpl_d, 50,50).lwm

In [24]:
niveauEmissionNmpn08, niveauEmissionNmpn08Plamade

(85.21881786033339, 100.96696024705047)

In [56]:
rqt="""
    SELECT t."IDTRONCON", t."TMJA_CBS", t."TMJAVLT", t."TMJAPLT", t."TMHVLD", t."TMHPLD", t."PCENTMPL", t."PCENTHPL",v."VITESSEVL", v."VITESSEPL"
 FROM echeance4."N_ROUTIER_TRAFIC" t JOIN echeance4."N_ROUTIER_VITESSE" v ON t."IDTRONCON"=v."IDTRONCON"
                                     JOIN echeance4."N_ROUTIER_TRONCON_L" tro ON t."IDTRONCON"=tro."IDTRONCON"
                                     JOIN echeance4."N_ROUTIER_ROUTE" r ON tro."IDROUTE"=r."IDROUTE"
 WHERE tro."CBS_GITT"
"""

with ct.ConnexionBdd('local_PlaMADE') as c : 
    dfDataAcoustique=pd.read_sql(rqt, c.sqlAlchemyConn)
dfDataAcoustique.columns=[c.lower() for c in dfDataAcoustique.columns]
dfDataAcoustique.drop_duplicates(['tmhvld','tmhpld'], inplace=True)

In [81]:
def trouverEcartRange(x) : 
    if 0 <= x <=2 : 
        return 'de 0 à 2'
    elif 2 < x <=4 :
        return 'de 2 à 4'
    elif 4 < x <=6 :
        return 'de 4 à 6'
    else : 
        return 'sup 6'

dfDataAcoustique['lwm_nmp08']=dfDataAcoustique.apply(lambda x : Emission.Route(x.tmhvld, x.tmhpld, 50,50).lwm if x.tmhpld>0 and x.tmhvld>0 else np.NaN, axis=1)
dfDataAcoustique['lwm_nmp08_plamade']=dfDataAcoustique.apply(lambda x : Emission.Route(x.tmhvld, (x.tmjaplt*x.pcentmpl)+(x.tmjaplt*x.pcenthpl), 50,50).lwm if 
                                                             ((x.tmhpld*x.pcentmpl)+(x.tmhpld*x.pcenthpl)) > 0 and x.tmhvld>0 else np.NaN, axis=1)
dfDataAcoustique['ecart']=abs(dfDataAcoustique.lwm_nmp08_plamade-dfDataAcoustique.lwm_nmp08)
dfDataAcoustique['ecart_range']=dfDataAcoustique.ecart.apply(lambda x : trouverEcartRange(x))
dfDataAcoustique['pc_pl']=dfDataAcoustique.tmjaplt/(dfDataAcoustique.tmjaplt+dfDataAcoustique.tmjavlt) *100
dfDataAcoustique=dfDataAcoustique.loc[~dfDataAcoustique.ecart.isna()]

In [82]:
dfDataAcoustique

,idtroncon,tmja_cbs,tmjavlt,tmjaplt,tmhvld,tmhpld,pcentmpl,pcenthpl,vitessevl,vitessepl,lwm_nmp08,lwm_nmp08_plamade,ecart,pc_pl,ecart_range
0,RTO2021024130018310.1671,12704,12369,335,745,22,1.13520,1.50480,50,50,80.522431,89.892842,9.370411,2.636965,sup 6
1,RTO2021047130018310.9190,13363,6408,274,386,18,1.76300,2.33700,50,50,78.131799,90.685608,12.553809,4.100569,sup 6
2,RTO2021047130018310.9182,11097,10664,433,643,28,1.67700,2.22300,80,80,80.267709,92.474615,12.206905,3.901955,sup 6
3,RTO2021064130018310.64405,18043,8464,558,510,36,2.65740,3.52260,50,50,79.919234,95.455989,15.536755,6.184881,sup 6
4,RTO2021086130018310.63914,15070,13920,1150,838,74,3.28090,4.34910,80,80,82.458603,99.484683,17.026080,7.631055,sup 6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52381,RTO2021033130018310.73885,14626,6884,429,429,29,3.36950,2.49050,50,50,79.100090,94.095117,14.995027,5.866266,sup 6
52497,RTO2021087130018310.4518,16622,15528,1094,968,73,3.78350,2.79650,50,50,82.809873,98.645216,15.835343,6.581639,sup 6
52873,RTO2021019130018310.55489,9754,9266,488,578,33,2.87500,2.12500,50,50,80.146932,93.999649,13.852717,5.003076,sup 6
52892,RTO2021019130018310.55524,9754,4633,244,289,17,2.87500,2.12500,50,50,77.178436,90.989349,13.810913,5.003076,sup 6


In [83]:
alt.Chart(dfDataAcoustique).transform_density(
    'ecart',
    as_=['ecart', 'density']
).mark_area().encode(
    x="ecart:Q",
    y='density:Q',
) | alt.Chart(dfDataAcoustique).transform_density(
    'ecart',
    as_=['ecart', 'density'],cumulative=True
).mark_area().encode(
    x="ecart:Q",
    y='density:Q',
)

alt.HConcatChart(...)

In [84]:
# interval selection in the scatter plot
pts = alt.selection(type="interval", encodings=["x"])

# left panel: scatter plot
points = alt.Chart().mark_point(filled=True).encode(
    x='pc_pl',
    y='ecart'
).transform_filter(
    pts
).properties(
    width=300,
    height=300
)

# right panel: histogram
mag = alt.Chart().mark_bar().encode(
    x=alt.X('mbin:N', title='ecart'),
    y="count()"
).properties(
    width=300,
    height=300
).add_selection(pts)

# build the chart:
alt.hconcat(
    points,
    mag,
    data=dfDataAcoustique
).transform_bin(
    "mbin",
    field="ecart",
    bin=alt.Bin(maxbins=20)
)

alt.HConcatChart(...)

,idtroncon,tmja_cbs,tmjavlt,tmjaplt,tmhvld,tmhpld,pcentmpl,pcenthpl,vitessevl,vitessepl,lwm_nmp08,lwm_nmp08_plamade,pc_pl,ecart
7978,RTO2021017130018310.36995,10133,5057,9,304,0,0.0731,0.0969,50,50,NaN,NaN,0.177655,NaN
9006,RTO2021033130018310.38898,8540,4261,9,257,0,0.0860,0.1140,50,50,NaN,NaN,0.210773,NaN
40476,RTO2020033528694052.58380,16617,11755,4862,8319,3151,-1.0000,-1.0000,130,90,96.197423,NaN,29.259192,NaN
40485,RTO2020040572139996.51241,1240,1170,70,80,-1,-1.0000,-1.0000,70,70,NaN,70.699754,5.645161,NaN
40486,RTO2020040572139996.51243,39760,14093,5187,848,283,-1.0000,-1.0000,130,90,85.871126,NaN,26.903527,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42360,RTO2020033572139996.58696,24180,10632,1400,650,76,-1.0000,-1.0000,130,90,81.909920,NaN,11.635638,NaN
42364,RTO2020017572139996.2907,31596,12228,1600,734,79,-1.0000,-1.0000,130,90,82.265278,NaN,11.570726,NaN
42411,RTO2020017572139996.2258,15202,7300,716,455,46,-1.0000,-1.0000,130,90,80.063008,NaN,8.932136,NaN
42412,RTO2020017572139996.2241,12456,5660,640,356,40,-1.0000,-1.0000,130,90,79.211454,NaN,10.158730,NaN


In [2]:
Emission.Route?

Init signature:
Emission.Route(
    debitVl,
    debitPl,
    vtsVl,
    vtsPl,
    categorieRevt='r2',
    ageRevt=10,
    allure='s',
    declivite=0,
    drainant=False,
)
Docstring:      fournir les information de puissance acoustique d'u troncon acoustiquement homogene d'une route 
Init docstring:
attributs a fournir : 
    categorieRevt : string : r1 ou r2 ou r3,
    ageRevt : integer, 
    vtsVl : integer entre 20 et 130, 
    vtsPl : integer entre 20 et 100, 
    declivite : integer entre -6 et 6 ,
    debitVl : integer >0 , 
    debitPl : integer >0 ,
    allure : string  : 's' ou 'a' ou 'd' defaut='s', 
    drainant : booleen defaut=false
attributs calcule : 
    lrwmVl : composante roulement vehicule unitaire VL
    lmwmVl : composante moteur vehicule unitaire VL
    lrwmPl : composante roulement vehicule unitaire PL
    lmwmPl : composante moteur vehicule unitaire PL
    lwmVl : emission vehicule unitaire VL
    lwmPl : emission vehicule unitaire PL
    lwVl : emission VL p